<a href="https://colab.research.google.com/github/youneslebboukh/text-summarization/blob/main/txt_summ_test2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"youneslbk","key":"ff30355568a25aef64e5c064a14e36fc"}'}

In [3]:
 !mkdir -p ~/.kaggle
 !cp kaggle.json ~/.kaggle/

In [4]:
  !chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle datasets download -d allen-institute-for-ai/CORD-19-research-challenge

100% 6.37G/6.38G [01:34<00:00, 78.3MB/s]
100% 6.38G/6.38G [01:34<00:00, 72.4MB/s]


In [6]:
!pip install zip_files

In [7]:
from zipfile import ZipFile
file_name="CORD-19-research-challenge.zip"
with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('Done')

Done


In [8]:
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt') # one time execution
import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
metadata = pd.read_csv("/content/metadata.csv")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,4,5,6,13,14,15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
metadata.head(3)

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
0,ug7v899j,d1aafb70c066a2068b02786f8929fd9c900897fb,PMC,Clinical features of culture-proven Mycoplasma...,10.1186/1471-2334-1-6,PMC35282,11472636,no-cc,OBJECTIVE: This retrospective chart review des...,2001-07-04,"Madani, Tariq A; Al-Ghamdi, Aisha A",BMC Infect Dis,NaN,NaN,NaN,document_parses/pdf_json/d1aafb70c066a2068b027...,document_parses/pmc_json/PMC35282.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,NaN
1,02tnwd4m,6b0567729c2143a66d737eb0a2f63f2dce2e5a7d,PMC,Nitric oxide: a pro-inflammatory mediator in l...,10.1186/rr14,PMC59543,11667967,no-cc,Inflammatory diseases of the respiratory tract...,2000-08-15,"Vliet, Albert van der; Eiserich, Jason P; Cros...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/6b0567729c2143a66d737...,document_parses/pmc_json/PMC59543.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
2,ejv2xln0,06ced00a5fc04215949aa72528f2eeaae1d58927,PMC,Surfactant protein-D and pulmonary host defense,10.1186/rr19,PMC59549,11667972,no-cc,Surfactant protein-D (SP-D) participates in th...,2000-08-25,"Crouch, Erika C",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/06ced00a5fc04215949aa...,document_parses/pmc_json/PMC59549.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN


In [9]:
import json
import glob

In [10]:
all_json = glob.glob(f'document_parses/*/*.json', recursive=True)
len(all_json)

245968

In [11]:
dic = {'paper_id': [], 'abstract': [], 'body_text': []}
for t in all_json:
  with open(t) as file: 
    content = json.load(file)
    paper_id = content['paper_id']
    abstract = []
    body_text = []
    if 'abstract' in content :
      if content['abstract']:
        # Abstract
        for entry in content['abstract']:
          abstract.append(entry['text'])
        # Body text
        for entry in content['body_text']:
          body_text.append(entry['text'])
        
        abstract = '\n'.join(abstract)
        body_text = '\n'.join(body_text)
        dic['paper_id'].append(paper_id)
        dic['abstract'].append(abstract)
        dic['body_text'].append(body_text)

In [12]:
data = pd.DataFrame(dic, columns=['paper_id', 'abstract', 'body_text'])

In [13]:
data.head()

,paper_id,abstract,body_text
0,b7083e4986d73a83a90c0a30806dc4f4d5793592,Non-viral gene delivery to skeletal muscle was...,Skeletal muscle can act as an effective platfo...
1,b50fcaa1eb0cecd4cb22356af1da97ee5598ff04,ADP-ribosylation is a protein modification res...,ADP-ribosylation is a reversible post-translat...
2,333a04fb2ab37297b4591e7d6179fd731f5c1bf3,Background: Canine distemper virus (CDV) is a ...,Canine distemper virus (CDV) is one of the mos...
3,15c7e413de2e615378e1c0d4fa23d1f3425af285,In the context of the ongoing Covid-19 pandemi...,I. EXECUTIVE SUMMARY Important note: This manu...
4,a600310d78ebe675d0d7bc5d9188a77d73e0e79a,The SARS outbreak in 2003 provides a unique op...,The SARS outbreak in 2003 provides a unique op...


In [14]:
import csv
import pandas as pd

In [15]:
def preprocessing(text):
    # remove mail
    text = re.sub(r'[a-z0-9._%+-]+@[a-z0-9.-]+\.[a-z]{2,}', 'MAIL', text)
    # remove doi
    text = re.sub(r'https\:\/\/doi\.org[^\s]+', 'DOI', text)
    # remove https
    text = re.sub(r'(\()?\s?http(s)?\:\/\/[^\)]+(\))?', '\g<1>LINK\g<3>', text)
    # remove single characters repeated at least 3 times for spacing error (e.g. s u m m a r y)
    text = re.sub(r'(\w\s+){3,}', ' ', text)
    # replace tags (e.g. [3] [4] [5]) with whitespace
    text = re.sub(r'(\[\d+\]\,?\s?){3,}(\.|\,)?', ' \g<2>', text)
    # replace tags (e.g. [3, 4, 5]) with whitespace
    text = re.sub(r'\[[\d\,\s]+\]', ' ',text)
     # replace tags (e.g. (NUM1) repeated at least 3 times with whitespace
    text = re.sub(r'(\(\d+\)\s){3,}', ' ',text)
    # replace '1.3' with '1,3' (we need it for split later)
    text = re.sub(r'(\d+)\.(\d+)', '\g<1>,\g<2>', text)
    # remove all full stops as abbreviations (e.g. i.e. cit. and so on)
    text = re.sub(r'\.(\s)?([^A-Z\s])', ' \g<1>\g<2>', text)
    # correctly spacing the tokens
    text = re.sub(r' {2,}', ' ', text)
    text = re.sub(r'\.{2,}', '.', text)
    text=re.sub(r"[<>()(,)|&©ø\[\]\'\";?~*!]", ' ', text) #remove <>()|&©ø"',;?~*!
    text=re.sub("(\\t)", ' ', text) #remove escape charecters
    text=re.sub("(\\r)", ' ', text) 
    text=re.sub("(\\n)", ' ', text)
    text= re.sub("(\s+)",' ',text) #remove multiple space
    # return lowercase text
    return text.lower()

In [16]:
data['prepro_body_text'] = data['body_text'].apply(preprocessing)

In [17]:
data.head()

,paper_id,abstract,body_text,prepro_body_text
0,b7083e4986d73a83a90c0a30806dc4f4d5793592,Non-viral gene delivery to skeletal muscle was...,Skeletal muscle can act as an effective platfo...,skeletal muscle can act as an effective platfo...
1,b50fcaa1eb0cecd4cb22356af1da97ee5598ff04,ADP-ribosylation is a protein modification res...,ADP-ribosylation is a reversible post-translat...,adp-ribosylation is a reversible post-translat...
2,333a04fb2ab37297b4591e7d6179fd731f5c1bf3,Background: Canine distemper virus (CDV) is a ...,Canine distemper virus (CDV) is one of the mos...,canine distemper virus cdv is one of the most ...
3,15c7e413de2e615378e1c0d4fa23d1f3425af285,In the context of the ongoing Covid-19 pandemi...,I. EXECUTIVE SUMMARY Important note: This manu...,i. executive summary important note: this manu...
4,a600310d78ebe675d0d7bc5d9188a77d73e0e79a,The SARS outbreak in 2003 provides a unique op...,The SARS outbreak in 2003 provides a unique op...,the sars outbreak in 2003 provides a unique op...


In [18]:
from nltk.tokenize import sent_tokenize

In [19]:
#sentences = []
sent = []
sent=sent_tokenize(data['prepro_body_text'][0])
#for s in data['prepro_body_text'][0]:
  #sent.append(sent_tokenize(s))
  #sent = [y for x in sent for y in x] # flatten list
  #sentences.append(sent)

In [20]:
print(sent)

['skeletal muscle can act as an effective platform for the long-term production and secretion of therapeutic proteins with systemic distribution and for the introduction of dna vaccines eliciting strong humoral and cellular immune responses for review see .', 'conversely the treatment of hereditary neuromuscular diseases is particularly challenging for non-viral vectors.', 'among issues are as follows: 1 the size of the muscle tissue which represents half of the total mass of the organism 2 the poor accessibility of profound muscles or peripheral nerves and 3 the progressive tissue remodeling along the natural history of some muscle diseases with active processes of necrosis/regeneration and fibrosis/lipidosis.', 's. braun afm-telethon evry cedex france e-mail: mail on the other hand non-viral vectors do bear interesting advantages over recombinant viruses.', 'non-viral vectors are made of plasmid dna naked or complexed to a variety of versatile molecules such as cationic lipids or pol

In [21]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [22]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [23]:
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

In [24]:
clean_sent = [remove_stopwords(r.split()) for r in sent]

In [25]:
print(clean_sent)

['skeletal muscle act effective platform long-term production secretion therapeutic proteins systemic distribution introduction dna vaccines eliciting strong humoral cellular immune responses review see .', 'conversely treatment hereditary neuromuscular diseases particularly challenging non-viral vectors.', 'among issues follows: 1 size muscle tissue represents half total mass organism 2 poor accessibility profound muscles peripheral nerves 3 progressive tissue remodeling along natural history muscle diseases active processes necrosis/regeneration fibrosis/lipidosis.', 's. braun afm-telethon evry cedex france e-mail: mail hand non-viral vectors bear interesting advantages recombinant viruses.', 'non-viral vectors made plasmid dna naked complexed variety versatile molecules cationic lipids polymers.', '1 well characterized structure fine-tuned 2 mostly nonimmunogenic provided carrying protein motifs.', 'allows repeated administrations chronic diseases 3 comparatively easy produce large 

In [26]:
#Download GloVe Word Embeddings
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip

--2020-12-26 19:35:13--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-12-26 19:35:13--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-12-26 19:35:14--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [27]:
# Extract word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [28]:
sent_vectors = []
for i in clean_sent:
  if len(i) != 0:
    v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
  else:
    v = np.zeros((100,))
  sent_vectors.append(v)

In [31]:
from nltk.cluster.util import cosine_distance
import networkx as nx
from sklearn.metrics.pairwise import cosine_similarity

In [32]:
sim_mat = np.zeros([len(clean_sent), len(clean_sent)])
for i in range(len(clean_sent)): 
      for j in range(len(clean_sent)): 
        if i != j: 
              sim_mat[i][j] = cosine_similarity (sent_vectors[i].reshape(1,100), sent_vectors[j].reshape(1,100))[0,0]

In [33]:
print( sim_mat)

[[0.         0.73532353 0.84382408 ... 0.87502355 0.86222625 0.64597001]
 [0.73532353 0.         0.71508312 ... 0.77627724 0.75654328 0.5265761 ]
 [0.84382408 0.71508312 0.         ... 0.87178745 0.85902152 0.64277553]
 ...
 [0.87502355 0.77627724 0.87178745 ... 0.         0.85805815 0.63963455]
 [0.86222625 0.75654328 0.85902152 ... 0.85805815 0.         0.72936708]
 [0.64597001 0.5265761  0.64277553 ... 0.63963455 0.72936708 0.        ]]


In [36]:
import networkx as nx 
nx_graph = nx.from_numpy_array(sim_mat) 
scores = nx.pagerank(nx_graph)

In [38]:
ranked_sentences = sorted(((scores[i],s) for i,s in 
                           enumerate(clean_sent)), reverse=True)
# Extract top 10 sentences as the summary 
for i in range(10): 
      print(ranked_sentences[i][1])

besides relatively large size many synthetic vectors often 150 nm main obstacles use synthetic complexes via systemic delivery aggregation instability toxicity including negatively charged naked plasmid dna polynucleotides delivered either directly combined physical methods ultrasound electroporation complexed various chemical entities cationic lipids polymers b uptake pathways involve either fusion muscle cell membrane- receptor- clathrin- caveolae- pinocytosis-dependent endocytosis.
furthermore also designed ignored contrast viral vectors non-viral gene transfer elicited large extent active intake processes.
nevertheless poor efficiency remains compatible applications require low levels therapeutic proteins genetic vaccines cancer peripheral limb ischemia table 9 1 .
therefore limb perfusion naked dna remains valid approach treat limb dystrophic muscles alternative viral vectors seropositive patients indications require large transgenes regional gene transfer .
today despite number s